In [83]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime as datetime
import os
import sys
import numpy as np

In [84]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'FPL')))


In [85]:
from FPL.fpl_api_collection import (
    get_bootstrap_data,
    get_current_gw,
    get_fixt_dfs,
    get_league_table
)
from FPL.fpl_utils import (
    define_sidebar,
    get_annot_size,
    map_float_to_color,
    get_text_color_from_hash,
    get_rotation
)
from FPL.fpl_params import (
    TIMEZONES_BY_CONTINENT,
    AUTHOR_CONTINENT,
    AUTHOR_CITY
)

In [86]:
team_fdr_df, team_fixt_df, team_ga_df, team_gf_df = get_fixt_dfs()

events_df = pd.DataFrame(get_bootstrap_data()['events'])

gw_min = min(events_df['id'])
gw_max = max(events_df['id'])

ct_gw = get_current_gw()

In [87]:
sui=team_fixt_df.reset_index()


val=team_fdr_df.reset_index()

In [88]:
# Rename the first column to 'Team'
sui.rename(columns={0: 'Team'}, inplace=True)
val.rename(columns={0: 'Team'}, inplace=True)

# Create new column names, keeping 'Team' as the first column
sui.columns = ['Team'] + [f'GW{col}' for col in range(1, len(sui.columns))]
val.columns = ['Team'] + [f'GW{col}' for col in range(1, len(val.columns))]

In [89]:
sui
val

,Team,GW1,GW2,GW3,GW4,GW5,GW6,GW7,GW8,GW9,...,GW29,GW30,GW31,GW32,GW33,GW34,GW35,GW36,GW37,GW38
0,ARS,2.0,4.0,2.0,4.0,5.0,2.0,2.0,3.0,5.0,...,4.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,2.0
1,AVL,2.0,5.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,...,5.0,3.0,3.0,2.0,3.0,5.0,2.0,3.0,4.0,3.0
2,BOU,2.0,3.0,2.0,4.0,5.0,2.0,2.0,5.0,4.0,...,2.0,2.0,2.0,2.0,2.0,3.0,5.0,3.0,5.0,2.0
3,BRE,2.0,5.0,2.0,5.0,4.0,2.0,2.0,3.0,2.0,...,3.0,4.0,4.0,5.0,2.0,2.0,3.0,2.0,2.0,2.0
4,BHA,2.0,3.0,5.0,2.0,3.0,4.0,4.0,4.0,2.0,...,5.0,3.0,2.0,2.0,3.0,2.0,3.0,2.0,5.0,4.0
5,CHE,5.0,2.0,2.0,3.0,2.0,2.0,3.0,5.0,3.0,...,5.0,4.0,3.0,2.0,3.0,2.0,5.0,4.0,3.0,2.0
6,CRY,3.0,2.0,4.0,2.0,3.0,2.0,5.0,2.0,4.0,...,4.0,2.0,2.0,5.0,2.0,5.0,3.0,4.0,2.0,5.0
7,EVE,2.0,4.0,2.0,4.0,2.0,2.0,3.0,2.0,2.0,...,2.0,5.0,5.0,2.0,5.0,4.0,2.0,3.0,2.0,4.0
8,FUL,3.0,2.0,2.0,2.0,3.0,2.0,5.0,3.0,2.0,...,4.0,5.0,5.0,3.0,4.0,2.0,4.0,2.0,3.0,5.0
9,IPS,5.0,5.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0,...,3.0,3.0,2.0,4.0,5.0,4.0,2.0,2.0,2.0,2.0


In [90]:
# Combine the two DataFrames for a complete list of teams
combined_teams = pd.concat([sui['Team'], val['Team']]).unique()

# Initialize an empty FDR matrix
fdr_matrix = pd.DataFrame(np.zeros((len(combined_teams), len(combined_teams))), 
                          index=combined_teams, 
                          columns=combined_teams)

# Populate the FDR matrix based on your criteria
# For simplicity, let's assume a mock FDR scoring based on upcoming matches:
# You can customize this part according to your actual difficulty ratings logic
for i in range(len(combined_teams)):
    for j in range(len(combined_teams)):
        if i != j:
            # Example scoring logic: 
            # Here we use a random number for demonstration. You might want to calculate it based on actual data.
            fdr_matrix.iloc[i, j] = np.random.randint(1, 6)  # Random score from 1 to 5

# Display the FDR matrix
print("FDR Matrix:")
print(fdr_matrix)

NameError: name 'np' is not defined